In [1]:
import pandas as pd

In [2]:
sepsis_df = pd.read_csv("../data/dataSepsis.csv", sep=";")

In [89]:
sepsis_df.head(1000)

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,...,WBC,Fibrinogen,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,isSepsis
0,103.0,90.0,NaN,NaN,NaN,NaN,30.0,NaN,21.0,45.0,...,5.7,NaN,317.0,83.14,0,NaN,NaN,-0.03,17,0
1,58.0,95.0,36.11,143.0,77.0,47.0,11.0,NaN,NaN,22.0,...,11.0,NaN,158.0,75.91,0,0.0,1.0,-98.60,10,0
2,91.0,94.0,38.50,133.0,74.0,48.0,34.0,NaN,NaN,31.0,...,8.3,NaN,465.0,45.82,0,1.0,0.0,-1195.71,11,0
3,92.0,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.0,...,12.0,NaN,298.0,52.01,1,1.0,0.0,-0.03,13,0
4,155.5,94.5,NaN,147.5,102.0,NaN,33.0,NaN,-12.0,13.0,...,9.3,NaN,26.0,64.24,1,1.0,0.0,-0.05,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,63.0,99.5,36.22,120.5,74.5,53.0,14.0,NaN,-5.0,NaN,...,NaN,NaN,84.0,39.06,1,0.0,1.0,-1188.85,8,1
996,109.0,92.0,NaN,118.0,80.0,NaN,NaN,NaN,NaN,38.0,...,6.8,NaN,299.0,52.67,1,1.0,0.0,-76.25,11,0
997,114.0,98.0,37.39,84.5,58.0,56.0,29.0,NaN,3.0,27.0,...,12.7,NaN,197.0,44.04,0,1.0,0.0,-37.48,8,0
998,115.0,97.5,37.89,115.0,68.0,NaN,15.5,NaN,NaN,NaN,...,NaN,NaN,NaN,40.60,1,0.0,1.0,-7.10,2,0


In [5]:
sepsis_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36302 entries, 0 to 36301
Data columns (total 41 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   HR                35506 non-null  float64
 1   O2Sat             34736 non-null  float64
 2   Temp              17101 non-null  float64
 3   SBP               34617 non-null  float64
 4   MAP               34846 non-null  float64
 5   DBP               27917 non-null  float64
 6   Resp              33890 non-null  float64
 7   EtCO2             1613 non-null   float64
 8   BaseExcess        7184 non-null   float64
 9   HCO3              16984 non-null  float64
 10  FiO2              6427 non-null   float64
 11  pH                9221 non-null   float64
 12  PaCO2             8430 non-null   float64
 13  SaO2              4335 non-null   float64
 14  AST               10391 non-null  float64
 15  BUN               32526 non-null  float64
 16  Alkalinephos      10269 non-null  float6

In [187]:
sepsis_df.drop(columns=['Unit1', 'Unit2'], inplace=True)

In [188]:
# very imbalanced dataset, needs resampling!
from collections import Counter
Counter(sepsis_df.isSepsis)

Counter({0: 33655, 1: 2647})

In [189]:
Counter(sepsis_df.Magnesium.isnull())

Counter({False: 27671, True: 8631})

In [190]:
2647/33655

0.07865101767939385

In [191]:
sepsis_df.Magnesium.isnull().value_counts()

False    27671
True      8631
Name: Magnesium, dtype: int64

## Get Value Counts (Null/Non-Null Per Column)

In [192]:
def col_val_counts(col_name):
    values = sepsis_df[col_name].isnull().value_counts()
    
    total = len(sepsis_df)
    non_null = values[0]
    null = total-non_null
    null_pctg = null/total
    non_null_pctg = non_null/total
    
    return {'Column':col_name,'Non_Null': values[0], 'Null': len(sepsis_df)-values[0], 
            'Null_Pctg':null_pctg, 'Non_Null_Pctg':non_null_pctg}

In [139]:
col_val_counts('Magnesium')['Non_Null']

27671

In [193]:
df_columns = list(sepsis_df.columns)

In [194]:
def col_vals_df(col_list):
    
    
#     return [{col_name:col_val_counts(col_name)} for col_name in copl_list]
    
#     for col_name in col_list:
#         print({col_name:col_val_counts(col_name)})
    
#     col_val_counts(col_name) for col_name in col_list
    
    
    return [col_val_counts(col_name) for col_name in col_list]

In [195]:
column_vals_df = pd.DataFrame(col_vals_df(df_columns))

### Checking Null Percentage:

Columns with high levels of null values will suffer more innaccuracy when imputing the mean.

For this project, I will keep columns with a minimum of 25% non-null values.


In [196]:
column_vals_df = column_vals_df.sort_values(by='Non_Null_Pctg', ascending=False).reset_index(drop=True)

In [197]:
column_vals_df

,Column,Non_Null,Null,Null_Pctg,Non_Null_Pctg
0,isSepsis,36302,0,0.000000,1.000000
1,Age,36302,0,0.000000,1.000000
2,ICULOS,36302,0,0.000000,1.000000
3,HospAdmTime,36302,0,0.000000,1.000000
4,Gender,36302,0,0.000000,1.000000
5,HR,35506,796,0.021927,0.978073
6,MAP,34846,1456,0.040108,0.959892
7,O2Sat,34736,1566,0.043138,0.956862
8,SBP,34617,1685,0.046416,0.953584
9,Resp,33890,2412,0.066443,0.933557


In [198]:
column_vals_with_threshold = column_vals_df[column_vals_df['Non_Null_Pctg']>0.25]

In [199]:
column_vals_with_threshold

,Column,Non_Null,Null,Null_Pctg,Non_Null_Pctg
0,isSepsis,36302,0,0.000000,1.000000
1,Age,36302,0,0.000000,1.000000
2,ICULOS,36302,0,0.000000,1.000000
3,HospAdmTime,36302,0,0.000000,1.000000
4,Gender,36302,0,0.000000,1.000000
5,HR,35506,796,0.021927,0.978073
6,MAP,34846,1456,0.040108,0.959892
7,O2Sat,34736,1566,0.043138,0.956862
8,SBP,34617,1685,0.046416,0.953584
9,Resp,33890,2412,0.066443,0.933557


## Imputing the Median Value 
For Null Values in Columns over 25% Non-Null Threshold

In [201]:
cols_to_impute = list(column_vals_with_threshold.Column)

In [203]:
len(cols_to_impute)

30

In [214]:
final_df = sepsis_df[cols_to_impute]

In [215]:
final_df

,isSepsis,Age,ICULOS,HospAdmTime,Gender,HR,MAP,O2Sat,SBP,Resp,...,Magnesium,Phosphate,Chloride,Temp,HCO3,PTT,AST,Bilirubin_total,Alkalinephos,pH
0,0,83.14,17,-0.03,0,103.0,NaN,90.0,NaN,30.0,...,2.0,3.3,85.0,NaN,45.0,NaN,16.0,0.3,98.0,7.37
1,0,75.91,10,-98.60,0,58.0,77.0,95.0,143.0,11.0,...,2.5,4.4,113.0,36.11,22.0,NaN,NaN,NaN,NaN,NaN
2,0,45.82,11,-1195.71,0,91.0,74.0,94.0,133.0,34.0,...,2.4,2.3,98.0,38.50,31.0,30.5,NaN,NaN,NaN,NaN
3,0,52.01,13,-0.03,1,92.0,NaN,100.0,NaN,NaN,...,NaN,NaN,111.0,NaN,29.0,NaN,NaN,NaN,NaN,NaN
4,0,64.24,2,-0.05,1,155.5,102.0,94.5,147.5,33.0,...,1.9,3.8,113.0,NaN,13.0,26.6,452.0,1.4,88.0,7.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36297,0,76.00,19,-14.90,1,59.0,110.0,95.0,154.0,14.0,...,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36298,0,84.00,25,-6.69,0,81.0,71.0,99.0,122.0,20.0,...,2.3,NaN,NaN,NaN,NaN,NaN,849.0,3.3,259.0,NaN
36299,0,30.00,2,-0.02,1,76.0,79.0,96.0,118.0,14.0,...,NaN,NaN,NaN,36.30,NaN,38.2,24.0,0.7,116.0,NaN
36300,0,60.00,15,-53.64,0,76.0,102.0,89.0,145.0,NaN,...,1.9,4.1,NaN,NaN,NaN,NaN,9.0,0.2,68.0,NaN


In [232]:
final_df.pH.isnull().values.any()

True

In [233]:
final_df.pH.median()

7.39

In [244]:
for col_name in list(final_df.columns):
    final_df[col_name].fillna(final_df[col_name].median(), inplace=True)

/Users/jasonkim/opt/anaconda3/envs/metis/lib/python3.8/site-packages/pandas/core/series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
/Users/jasonkim/opt/anaconda3/envs/metis/lib/python3.8/site-packages/pandas/core/series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
/Users/jasonkim/opt/anaconda3/envs/metis/lib/python3.8/site-packages/pandas/core/series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

In [245]:
final_df

,isSepsis,Age,ICULOS,HospAdmTime,Gender,HR,MAP,O2Sat,SBP,Resp,...,Magnesium,Phosphate,Chloride,Temp,HCO3,PTT,AST,Bilirubin_total,Alkalinephos,pH
0,0,83.14,17,-0.03,0,103.0,80.0,90.0,120.0,30.0,...,2.0,3.3,85.0,36.80,45.0,30.7,16.0,0.3,98.0,7.37
1,0,75.91,10,-98.60,0,58.0,77.0,95.0,143.0,11.0,...,2.5,4.4,113.0,36.11,22.0,30.7,32.0,0.8,72.0,7.39
2,0,45.82,11,-1195.71,0,91.0,74.0,94.0,133.0,34.0,...,2.4,2.3,98.0,38.50,31.0,30.5,32.0,0.8,72.0,7.39
3,0,52.01,13,-0.03,1,92.0,80.0,100.0,120.0,18.0,...,2.0,3.4,111.0,36.80,29.0,30.7,32.0,0.8,72.0,7.39
4,0,64.24,2,-0.05,1,155.5,102.0,94.5,147.5,33.0,...,1.9,3.8,113.0,36.80,13.0,26.6,452.0,1.4,88.0,7.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36297,0,76.00,19,-14.90,1,59.0,110.0,95.0,154.0,14.0,...,2.0,3.0,106.0,36.80,24.0,30.7,32.0,0.8,72.0,7.39
36298,0,84.00,25,-6.69,0,81.0,71.0,99.0,122.0,20.0,...,2.3,3.4,106.0,36.80,24.0,30.7,849.0,3.3,259.0,7.39
36299,0,30.00,2,-0.02,1,76.0,79.0,96.0,118.0,14.0,...,2.0,3.4,106.0,36.30,24.0,38.2,24.0,0.7,116.0,7.39
36300,0,60.00,15,-53.64,0,76.0,102.0,89.0,145.0,18.0,...,1.9,4.1,106.0,36.80,24.0,30.7,9.0,0.2,68.0,7.39


In [247]:
final_df.pH.isnull().values.any()

False

In [248]:
final_df.isnull().values.any()

False

In [249]:
final_df.to_pickle("sepsis_df_median_imputed.pkl")